In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.explode

In [2]:
    val conf=new SparkConf()
      .setMaster("local")//启动本地化计算
      .setAppName("json")//设置本程序名称

conf = org.apache.spark.SparkConf@41ac9db6


org.apache.spark.SparkConf@41ac9db6

In [3]:
    val sc=new SparkContext(conf)
    val spark = SparkSession.builder().appName("json_app").getOrCreate()

sc = org.apache.spark.SparkContext@71f7443
spark = org.apache.spark.sql.SparkSession@2811a303


org.apache.spark.sql.SparkSession@2811a303

In [4]:
 val data=spark.read.option("multiLine", true).option("mode", "PERMISSIVE").json("../data/test.json")//读取本地文件


data = [metaDataInfo: struct<Export time: string, Exported by: string ... 2 more fields>, pageSize: bigint ... 5 more fields]


[metaDataInfo: struct<Export time: string, Exported by: string ... 2 more fields>, pageSize: bigint ... 5 more fields]

In [5]:
data.show

+--------------------+--------+--------------------+-------------+----------+----------+----------+
|        metaDataInfo|pageSize|            policies|  queryTimeMS|resultSize|startIndex|totalCount|
+--------------------+--------+--------------------+-------------+----------+----------+----------+
|[Mar 21, 2018 6:3...|       0|[[[], [], [], [],...|1521614211412|         0|         0|         0|
+--------------------+--------+--------------------+-------------+----------+----------+----------+



In [6]:
data.select(explode(data("policies"))).show

+--------------------+
|                 col|
+--------------------+
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
+--------------------+



In [25]:
var jsonDF = data.select(explode(data("policies"))).toDF("policies")
jsonDF.show

+--------------------+
|            policies|
+--------------------+
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
|[[], [], [], [], ...|
+--------------------+



jsonDF = [policies: struct<allowExceptions: array<string>, dataMaskPolicyItems: array<string> ... 17 more fields>]


[policies: struct<allowExceptions: array<string>, dataMaskPolicyItems: array<string> ... 17 more fields>]

In [26]:
jsonDF=jsonDF.select(jsonDF("policies.service"), explode(jsonDF("policies.policyItems.users"))).toDF("service","users")


jsonDF = [service: string, users: array<string>]


[service: string, users: array<string>]

In [27]:
jsonDF.show

+--------------------+---------------+
|             service|          users|
+--------------------+---------------+
|Atlas_service_new...|[dhaval, admin]|
|Atlas_service_new...|[dhaval, admin]|
|Atlas_service_new...|[dhaval, admin]|
|        hdfs service|       [dhaval]|
|        hdfs service|     [keyadmin]|
|     hdfs service 2d|       [dhaval]|
|     hdfs service 2d|     [keyadmin]|
+--------------------+---------------+



In [21]:
jsonDF = jsonDF.select(jsonDF("service"), explode(jsonDF("users"))).toDF("service","users").distinct
jsonDF.show

+--------------------+--------+
|             service|   users|
+--------------------+--------+
|     hdfs service 2d|keyadmin|
|Atlas_service_new...|  dhaval|
|     hdfs service 2d|  dhaval|
|        hdfs service|keyadmin|
|Atlas_service_new...|   admin|
|        hdfs service|  dhaval|
+--------------------+--------+



jsonDF = [service: string, users: string]


[service: string, users: string]